#@title 🚀 蒼狼 AI V2.2 可觀測性分析平台 - Colab 一鍵啟動
#@markdown ---
#@markdown 👋 **歡迎使用！** 此 Colab 筆記本將引導您快速設定並啟動「蒼狼 AI V2.2」應用程式的後端與前端服務。
#@markdown 
#@markdown **👇 請依序設定以下參數，然後點擊左側的「播放」按鈕執行儲存格以啟動服務。**
#@markdown 
#@markdown ---
#@markdown ### 1. 專案與程式碼設定
#@markdown **程式碼分支 (`branch_name`)**: 指定要從 GitHub 下載的程式碼分支。
#@markdown - `main`: 官方穩定發布版 (建議)。
#@markdown - `dev`: 開發整合分支。
#@markdown - `feat/...`: 特定新功能開發分支。
branch_name = "main" #@param {type:"string"}
#@markdown 
#@markdown ---
#@markdown ### 2. 操作模式設定
#@markdown **操作模式 (`operation_mode`)**: 選擇應用程式的資料儲存方式。
#@markdown - **暫存模式 (transient)**：無需 Google Drive 授權，服務將使用臨時儲存空間，所有資料在 Colab 會話結束後將會遺失。適合快速體驗或測試。
#@markdown - **持久模式 (persistent)**：需要授權掛載您的 Google Drive，應用程式資料 (如資料庫檔案) 將儲存在您的 Drive 中，實現資料持久化。建議用於實際使用。
operation_mode = "transient" #@param ["transient", "persistent"]
#@markdown 
#@markdown <font color="red">**⚠️ 重要提示 (持久模式):**</font> 若選擇「持久模式」，請務必在執行此 Colab 前，已在左側「🔑 密鑰」管理員中設定好所有必要的 Secrets (如 `COLAB_GOOGLE_API_KEY`, `GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT`, `WOLF_IN_FOLDER_ID`, `WOLF_PROCESSED_FOLDER_ID`)
#@markdown ---
#@markdown ### 3. 啟動模式設定
#@markdown **啟動模式 (`MODE`)**: 選擇應用程式日誌的輸出方式。
#@markdown - **一般模式 (normal)**：應用程式日誌將寫入到專案資料夾內的日誌檔案 (`backend.log`, `frontend.log`)。適合常規運行。
#@markdown - **除錯模式 (debug)**：應用程式日誌將直接混合輸出到此 Colab 儲存格的輸出區域，方便即時追蹤和除錯。
MODE = "normal" #@param ["normal", "debug"]
#@markdown 
#@markdown ---
#@markdown ✅ 設定完成後，請執行此儲存格以開始部署和啟動程序。

In [ ]:
import os
import shutil
import time
import requests
from IPython.display import display, HTML, clear_output

clear_output(wait=True)
print(f"您選擇的操作模式是：{operation_mode}")
print(f"您選擇的啟動模式是：{MODE}")

os.environ['OPERATION_MODE'] = operation_mode
print(f"環境變數 OPERATION_MODE 已設定為: {os.getenv('OPERATION_MODE')}")

print("INFO: 開始執行 Colab 啟動程序...")

print("\nSTEP 1: 檢查是否需要掛載 Google Drive...")
if operation_mode == "persistent":
    print("INFO: 持久模式 - 開始掛載 Google Drive...")
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        print("SUCCESS: Google Drive 已成功掛載到 /content/drive")
    except Exception as e:
        print(f"ERROR: 掛載 Google Drive 失敗: {e}")
        raise SystemExit("Google Drive 掛載失敗，請處理後重試。")
else:
    print("INFO: 暫存模式 - 跳過 Google Drive 掛載。")

print("\nSTEP 2 & 3: Colab Secrets 設定引導與檢查...")
all_secrets_ok = True
if not os.getenv('COLAB_GOOGLE_API_KEY'):
    print("  ⚠️ COLAB_GOOGLE_API_KEY 未設定。AI 分析功能可能無法使用。")
else:
    print("  ✅ COLAB_GOOGLE_API_KEY 已設定。")

if operation_mode == "persistent":
    print("\nINFO: 持久模式 - 檢查 Google Drive 相關 Colab Secrets...")
    secrets_to_check_persistent = ['GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT', 'WOLF_IN_FOLDER_ID', 'WOLF_PROCESSED_FOLDER_ID']
    missing_secrets_persistent = [s for s in secrets_to_check_persistent if not os.getenv(s)]
    if missing_secrets_persistent:
        all_secrets_ok = False
        for secret_name in missing_secrets_persistent:
            print(f"  ❌ {secret_name} 未設定或為空值！")
    else:
        print("  ✅ 持久模式所需的所有 Secrets 均已設定。")
else:
    print("\nINFO: 暫存模式 - 跳過 Google Drive 相關 Colab Secrets 的檢查。")

if not all_secrets_ok:
    raise SystemExit("❌ 錯誤：持久模式下，必要的 Colab Secrets 設定不完整，無法繼續。")
else:
    print("\n✅ Colab Secrets 設定檢查完畢。")

print("\nSTEP 4: 複製專案程式碼...")
GIT_REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git"
PROJECT_PARENT_DIR = "/content/wolf_project"
PROJECT_DIR_NAME = "wolfAI_v1"
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)

if os.path.exists(FULL_PROJECT_PATH):
    shutil.rmtree(FULL_PROJECT_PATH)
os.makedirs(PROJECT_PARENT_DIR, exist_ok=True)
clone_command = f"git clone --depth 1 -b {branch_name} {GIT_REPO_URL} {FULL_PROJECT_PATH}"
clone_status = os.system(clone_command)

if clone_status != 0 or not os.path.isdir(FULL_PROJECT_PATH):
    raise SystemExit(f"ERROR: 專案複製失敗。請檢查 URL ('{GIT_REPO_URL}') 和分支 ('{branch_name}')。")
else:
    print(f"SUCCESS: 專案已成功複製到 '{FULL_PROJECT_PATH}'。")

print("\nSTEP 5: 執行主控啟動腳本 scripts/start.sh...")
start_script_path = os.path.join(FULL_PROJECT_PATH, 'scripts', 'start.sh')
if os.path.exists(start_script_path):
    os.system(f"chmod +x {start_script_path}")
    start_script_command = f"{start_script_path} --mode={MODE}"
    exit_code = os.system(start_script_command)
    if exit_code != 0:
        print(f"ERROR: 啟動腳本執行失敗，返回碼: {exit_code}。請檢查日誌。")
else:
    raise SystemExit(f"ERROR: 主控啟動腳本 {start_script_path} 未找到！")

print("\nSTEP 6: 健康度偵測 (Health Check) - 請耐心等待約 1-2 分鐘...")
BACKEND_SERVICE_PORT = 8000
FRONTEND_SERVICE_PORT = 3000
max_wait_duration_seconds = 120
check_interval_seconds = 10
start_check_time = time.time()
backend_service_ready, frontend_service_ready = False, False

while time.time() - start_check_time < max_wait_duration_seconds:
    try:
        if not backend_service_ready and requests.get(f"http://localhost:{BACKEND_SERVICE_PORT}/api/health", timeout=5).status_code == 200:
            backend_service_ready = True
            print(f"[{time.strftime('%H:%M:%S')}] ✅ 後端服務 READY.")
    except requests.RequestException:
        print(f"[{time.strftime('%H:%M:%S')}] ⏳ 後端服務未就緒...")

    try:
        if not frontend_service_ready and requests.get(f"http://localhost:{FRONTEND_SERVICE_PORT}", timeout=5).status_code == 200:
            frontend_service_ready = True
            print(f"[{time.strftime('%H:%M:%S')}] ✅ 前端服務 READY.")
    except requests.RequestException:
        print(f"[{time.strftime('%H:%M:%S')}] ⏳ 前端服務未就緒...")

    if backend_service_ready and frontend_service_ready:
        print("\n🎉 太棒了！後端和前端服務均已成功啟動！")
        break

    if time.time() - start_check_time >= max_wait_duration_seconds:
        print("\n⏱️ 服務啟動或健康檢查超時。")
        break

    time.sleep(check_interval_seconds)

print("\nSTEP 7: 顯示結果...")
if backend_service_ready and frontend_service_ready:
    border = "="*70
    print(f"\n{border}")
    display(HTML("<h2>✅ 應用程式已就緒，可以開始使用了！</h2>"))
    print("🔗 前端應用程式訪問網址 (Colab Proxy URL):")
    print(f"   請在本儲存格的輸出區域上方尋找由 Colab 自動生成的、形如 https://*.googleusercontent.com/ 的公開連結。")
    print(f"{border}\n")
else:
    print("\n❌ 很抱歉，一個或多個服務未能成功啟動。")
    print("   請仔細檢查上方的日誌輸出以了解詳細錯誤信息。")

print("INFO: Colab 啟動程序執行完畢。")